In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["OPENAI_API_KEY"] 

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
# txt_file_path = "scalexi.txt"
# txt_file_path = "tenacademy.txt"
# txt_file_path = "amharic.txt"
txt_file_path = "pbx.txt"

In [5]:
loader = TextLoader(file_path= txt_file_path, encoding="utf-8",)
data = loader.load()

In [25]:
!pip install torch
!pip install transformers

In [28]:
from transformers import AutoTokenizer

amharic_tokenizer = AutoTokenizer.from_pretrained("BiniyamAjaw/amharic_tokenizer")

tokenizer_config.json: 100%|██████████| 146/146 [00:00<00:00, 39.5kB/s]
tokenizer.json:   0%|          | 0.00/5.25M [00:00<?, ?B/s]

tokenizer.json: 100%|██████████| 5.25M/5.25M [00:00<00:00, 39.7MB/s]
special_tokens_map.json: 100%|██████████| 27.0/27.0 [00:00<00:00, 31.1kB/s]


In [30]:
amharic_tokenizer.tokenize()

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=3100, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [8]:
embeddings = OpenAIEmbeddings()

/home/lillian_assefa/.conda/envs/python3.8/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [10]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4",max_tokens=500)

/home/lillian_assefa/.conda/envs/python3.8/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [12]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [13]:
query = "what is the text about?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian_assefa/.conda/envs/python3.8/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The text appears to be a table of contents or index from a user manual or guide. It outlines sections on making calls, managing contacts, using hot desking (including logging in and out), third-party call control with the Rainbow application, and programming your telephone. The manual or guide is copyrighted by ALE in 2020.'

In [14]:
query = "How can I assign a speed dial on it"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"To assign a speed dial, do the following:\n\n1. Long press on a free programmable key or go to Menu -> Features -> Programmable Key.\n\n2. Use up-down and 'ok' navigation keys to select: Programmable Key.\n\n3. Create a speed dial key.\n\n4. Define the type of programmable key: Speed dial.\n\n5. Associate a SIP account to use to make the call.\n\n6. Enter the number and its label.\n\n7. Save the speed dial key.\n\nRemember, you can make the call using speed dial by selecting the key associated with the contact."

In [56]:
query = "What is the project about"
result = conversation_chain({"question": query})
answer = result["answer"]
answer


"The project is about developing an AI solution for AiQEM, an African startup focused on AI and Blockchain business solutions. AiQEM's latest project is an AI-based Telegram Ad solution called Adbar. The goal of the project is to improve the effectiveness of their promotional efforts by integrating powerful AI capabilities for Amharic text manipulation. \n\nThe project aims to create an Amharic RAG (Retrieval Augmented Generation) pipeline that will generate Amharic-based creative text Ad contents based on campaign information such as brand and product information and the content history of a Telegram channel. A successfully delivered project will ensure that the advertisements are both catchy and relevant to the Telegram community. \n\nThis will be achieved by finetuning a Language Learning Model (LLM) that has the capability to embed Amharic texts. The model should be selected from suitable open-source LLM models already capable of embedding Amharic texts, such as Nous Hermes Mistral

In [58]:
query = "How many deliverables are there?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer


'There are two deliverables for this project:\n\n1. Interim Submission - Due Wednesday 8pm UTC\n2. Final Submission - Due Saturday 8pm UTC'

In [15]:
from IPython.display import display, Markdown

formatted_answer = f"**Answer:** {answer}"
display(Markdown(formatted_answer))

**Answer:** To assign a speed dial, do the following:

1. Long press on a free programmable key or go to Menu -> Features -> Programmable Key.

2. Use up-down and 'ok' navigation keys to select: Programmable Key.

3. Create a speed dial key.

4. Define the type of programmable key: Speed dial.

5. Associate a SIP account to use to make the call.

6. Enter the number and its label.

7. Save the speed dial key.

Remember, you can make the call using speed dial by selecting the key associated with the contact.